In [ ]:
from datasets import load_dataset

ds = load_dataset("allenai/WildChat-4.8M")


In [ ]:
from datasets import load_dataset  
print(ds["train"].features) 
print(ds["train"].column_names)

## 1. only "english" and filter out those data without content including "literature review"

In [ ]:
from datasets import load_dataset

train = ds["train"]

def is_english(example):
    lang = example.get("language") or ""
    return isinstance(lang, str) and lang.lower() == "english"

def contains_lit_review(example):
    conv = example.get("conversation") or example.get("conversations") or []
    if isinstance(conv, dict):
        content = conv.get("content") or conv.get("text") or ""
        return isinstance(content, str) and "literature review" in content.lower()
    for msg in conv:
        if isinstance(msg, str):
            if "literature review" in msg.lower():
                return True
        if isinstance(msg, dict):
            content = msg.get("content")
            if isinstance(content, str) and "literature review" in content.lower():
                return True
    return False

english_ds = train.filter(is_english)

result_ds = english_ds.filter(contains_lit_review)

print("matches:", result_ds.num_rows)
print(result_ds[:5]) 

In [ ]:
from datasets import load_from_disk
import os

output_dir = r"C:\Users\25811\Desktop\Summer Search\Yu\Data_ds_format"
os.makedirs(output_dir, exist_ok=True)

result_ds.save_to_disk(output_dir)


In [ ]:
print(ds)                       
print(ds["train"].features)     
print(ds["train"].column_names) 

sample = ds["train"][0]
print(type(sample.get("conversation")), sample.get("conversation"))

In [ ]:
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

try:
    from dateutil import parser as _dateutil_parser
except Exception:
    _dateutil_parser = None

ts_col = "timestamp"   
batch_size = 10000

def as_datetime(v):
    if v is None:
        return None
    if isinstance(v, datetime):
        return v
    if isinstance(v, (int, float)):
        return datetime.fromtimestamp(v/1000.0) if abs(v) > 1e11 else datetime.fromtimestamp(v)
    if isinstance(v, str):
        try:
            return datetime.fromisoformat(v)
        except Exception:
            if _dateutil_parser:
                try:
                    return _dateutil_parser.parse(v)
                except Exception:
                    return None
            return None
    return None


counter = Counter()
n = len(result_ds)
for start in range(0, n, batch_size):
    end = min(n, start + batch_size)
    batch = result_ds[start:end]
    vals = batch[ts_col]
    for v in vals:
        if isinstance(v, (list, tuple)):
            for sub in v:
                dt = as_datetime(sub)
                if dt:
                    counter[dt.date()] += 1
        else:
            dt = as_datetime(v)
            if dt:
                counter[dt.date()] += 1


dates_sorted = sorted(counter.keys())
counts = [counter[d] for d in dates_sorted]

plt.figure(figsize=(12,4))
plt.plot(dates_sorted, counts, marker='o', linestyle='-')
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation=45)
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Daily distribution of timestamps')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Simple moving average
def moving_average(y, window):
    kernel = np.ones(window) / window
    return np.convolve(y, kernel, mode="same")

window = 7   # 7 / 14 / 30
smoothed = moving_average(counts, window)

plt.figure(figsize=(12,4))
plt.plot(dates_sorted, counts, alpha=0.4, label="raw")
plt.plot(dates_sorted, smoothed, color="red", linewidth=2, label=f"MA {window}")
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation=45); plt.legend()
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Daily distribution of timestamps')
plt.tight_layout()
plt.show()

In [ ]:
sample = result_ds[0]
print(type(sample.get("model")), sample.get("model"))
print(result_ds.unique("model"))
print(type(sample.get("turn")), sample.get("turn"))

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

def model_family(name):
    if not name: 
        return "other"
    n = name.lower()
    if n.startswith("gpt-4o"):
        return "gpt-4o"
    if n.startswith("gpt-3.5"):
        return "gpt-3.5-turbo"
    if n.startswith("gpt-4.1"):
        return "gpt-4.1-mini"
    if n.startswith("gpt-4"):
        return "gpt-4"
    if n.startswith("o1"):
        return "o1"
    return "other"

cnt = Counter()
batch_size = 10000
n = len(result_ds)
for start in range(0, n, batch_size):
    vals = result_ds[start:start+batch_size]["model"]   
    cnt.update(model_family(v) for v in vals)

print("counts:", dict(cnt))

labels, values = zip(*cnt.most_common())
plt.figure(figsize=(8,4))
plt.bar(labels, values, color="C0")
plt.xticks(rotation=45, ha="right")
plt.title("Model family counts")
plt.tight_layout()
plt.show()


## Model * Turns

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

def model_family5(name):
    if not name:
        return None
    n = name.lower()
    if n.startswith("gpt-4o"):
        return "gpt-4o"
    if n.startswith("gpt-3.5"):
        return "gpt-3.5-turbo"
    if n.startswith("gpt-4.1"):
        return "gpt-4.1-mini"
    if n.startswith("gpt-4"):
        return "gpt-4"
    if n.startswith("o1"):
        return "o1"
    return None

models = result_ds["model"]
turns  = result_ds["turn"]

samples = defaultdict(list)
for m, t in zip(models, turns):
    fam = model_family5(m)
    if fam is None:
        continue
    try:
        val = int(t)
    except Exception:
        continue
    samples[fam].append(val)

order = ["gpt-4o", "gpt-3.5-turbo", "gpt-4.1-mini", "gpt-4", "o1"]
data = []
labels = []
for fam in order:
    lst = samples.get(fam, [])
    if not lst:
        continue
    data.append(lst)
    labels.append(f"{fam}\n(n={len(lst)})")

if not data:
    raise RuntimeError("Fail to collect turn data")

plt.figure(figsize=(12,6))
plt.boxplot(data, tick_labels=labels, showfliers=False)
plt.ylabel("turn (int)")
plt.title("Distribution of 'turn' by model family")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

In [ ]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import random

def model_family5(name):
    if not name:
        return None
    n = name.lower()
    if n.startswith("gpt-4o"):
        return "gpt-4o"
    if n.startswith("gpt-3.5"):
        return "gpt-3.5-turbo"
    if n.startswith("gpt-4.1"):
        return "gpt-4.1-mini"
    if n.startswith("gpt-4"):
        return "gpt-4"
    if n.startswith("o1"):
        return "o1"
    return None

models = result_ds["model"]
turns  = result_ds["turn"]
samples = defaultdict(list)
for m, t in zip(models, turns):
    fam = model_family5(m)
    if fam is None:
        continue
    try:
        val = int(t)
    except Exception:
        continue
    samples[fam].append(val)

order = ["gpt-4o", "gpt-3.5-turbo", "gpt-4.1-mini", "gpt-4", "o1"]

# compute and print robust stats
print("family\tcount\tmedian\tIQR\t10th\t90th\tmean")
for fam in order:
    lst = np.array(samples.get(fam, []))
    if lst.size == 0:
        continue
    med = np.median(lst)
    q1, q3 = np.percentile(lst, [25, 75])
    iqr = q3 - q1
    p10, p90 = np.percentile(lst, [10, 90])
    mean = np.mean(lst)
    print(f"{fam}\t{len(lst)}\t{med:.2f}\t{iqr:.2f}\t{p10:.2f}\t{p90:.2f}\t{mean:.2f}")

# 1) boxplot on log1p scale (recommended for skewed counts)
data_log = [np.log1p(samples[f]) for f in order if samples.get(f)]
labels = [f"{f}\n(n={len(samples[f])})" for f in order if samples.get(f)]

plt.figure(figsize=(10,5))
try:
    plt.boxplot(data_log, tick_labels=labels, showfliers=False)
except TypeError:
    plt.boxplot(data_log, labels=labels, showfliers=False)
plt.ylabel("log1p(turn)")
plt.title("Boxplot (log1p scale) of 'turn' by model family")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# 2) violin + jittered scatter on original scale (shows raw spread)
data = [samples[f] for f in order if samples.get(f)]
pos = np.arange(1, len(data) + 1)

plt.figure(figsize=(12,5))
# violin (matplotlib)
parts = plt.violinplot(data, positions=pos, showmeans=False, showextrema=True)
for pc in parts['bodies']:
    pc.set_alpha(0.6)
# jittered scatter
for i, lst in enumerate(data, start=1):
    y = np.array(lst)
    x = np.random.normal(i, 0.08, size=y.size)  # small jitter
    plt.scatter(x, y, s=5, alpha=0.3, color='k')

plt.xticks(pos, labels, rotation=45, ha="right")
plt.ylabel("turn (int)")
plt.title("Violin + jittered points (original scale)")
plt.tight_layout()
plt.show()

In [ ]:
# A: ignore turn > cutoff
cutoff = 40
filtered = {f: [v for v in samples.get(f, []) if v <= cutoff] for f in order}
removed_counts = {f: len(samples.get(f, [])) - len(filtered[f]) for f in order}
print("removed (per family) when cutoff=%d:" % cutoff, removed_counts)

data = [filtered[f] for f in order if filtered.get(f)]
labels = [f"{f}\n(n={len(samples.get(f,[]))})" for f in order if filtered.get(f)]

plt.figure(figsize=(12,5))
parts = plt.violinplot(data, positions=np.arange(1, len(data)+1), showmeans=False, showextrema=True)
for pc in parts['bodies']:
    pc.set_alpha(0.6)
# jitter points
for i, lst in enumerate(data, start=1):
    y = np.array(lst)
    if y.size == 0:
        continue
    x = np.random.normal(i, 0.06, size=y.size)
    plt.scatter(x, y, s=8, alpha=0.6, color='k')
plt.xticks(np.arange(1, len(data)+1), labels, rotation=45, ha='right')
plt.ylabel("turn (int)")
plt.title(f"Violin + jitter (turn <= {cutoff})")
plt.ylim(-0.5, cutoff + 2)
plt.tight_layout()
plt.show()

# B: log scale
data_all = [samples[f] for f in order if samples.get(f)]
labels_all = [f"{f}\n(n={len(samples.get(f,[]))})" for f in order if samples.get(f)]

plt.figure(figsize=(12,5))
parts = plt.violinplot(data_all, positions=np.arange(1, len(data_all)+1), showmeans=False, showextrema=True)
for pc in parts['bodies']:
    pc.set_alpha(0.6)
for i, lst in enumerate(data_all, start=1):
    y = np.array(lst)
    if y.size == 0:
        continue
    x = np.random.normal(i, 0.06, size=y.size)
    plt.scatter(x, y, s=6, alpha=0.5, color='k')
plt.xticks(np.arange(1, len(data_all)+1), labels_all, rotation=45, ha='right')
plt.yscale('log')
plt.ylabel("turn (int), log scale")
plt.title("Violin + jitter (all points, log y-axis)")
plt.tight_layout()
plt.show()

## Analysis With Characterist numbers

In [ ]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

def extract_contents(conv):
    if conv is None:
        return []
    if isinstance(conv, (list, tuple)):
        texts = []
        for msg in conv:
            if isinstance(msg, str):
                texts.append(msg)
            elif isinstance(msg, dict):
                c = msg.get("content") or msg.get("text")
                if isinstance(c, str):
                    texts.append(c)
        return texts
    return []

def count_words_from_text(s):
    if not s:
        return 0
    return len(s.split())

models = result_ds["model"]
turns  = result_ds["turn"]
convs  = result_ds["conversation"]

avg_words_per_turn = []
skipped = 0
for m, t, conv in zip(models, turns, convs):
    try:
        turn_val = int(t)
    except Exception:
        skipped += 1
        continue
    if turn_val <= 0:
        skipped += 1
        continue
    texts = extract_contents(conv)
    total_words = sum(count_words_from_text(x) for x in texts)
    avg = total_words / turn_val
    avg_words_per_turn.append(avg)

avg_arr = np.array(avg_words_per_turn)
print(f"kept samples: {avg_arr.size}, skipped (bad turn/zero): {skipped}")

if avg_arr.size:
    med = np.median(avg_arr)
    q1, q3 = np.percentile(avg_arr, [25,75])
    iqr = q3 - q1
    p10, p90 = np.percentile(avg_arr, [10,90])
    mean = np.mean(avg_arr)
    print("metric\tcount\tmedian\tIQR\t10th\t90th\tmean")
    print(f"avg_words_per_turn\t{avg_arr.size}\t{med:.2f}\t{iqr:.2f}\t{p10:.2f}\t{p90:.2f}\t{mean:.2f}")
else:
    raise RuntimeError("no valid avg_words_per_turn data")

In [ ]:
# plot 1: histplot
plt.figure(figsize=(8,4))
plt.hist(avg_arr, bins=40, color="C0", alpha=0.8)
plt.xlabel("avg words per turn")
plt.ylabel("count")
plt.title("Histogram of avg words per turn")
plt.tight_layout()
plt.show()

# plot 2: boxplot（log1p）
# plt.figure(figsize=(6,4))
# try:
#     plt.boxplot([np.log1p(avg_arr)], tick_labels=["log1p(avg words/turn)"], showfliers=False)
# except TypeError:
#     plt.boxplot([np.log1p(avg_arr)], labels=["log1p(avg words/turn)"], showfliers=False)
# plt.title("Boxplot (log1p) of avg words per turn")
# plt.tight_layout()
# plt.show()

# plot 3: violin + jitter 
# cutoff = 5000.0  
# vis_vals = avg_arr[avg_arr <= cutoff]
# print(f"visualizing {vis_vals.size} points (<= {cutoff}), removed {avg_arr.size - vis_vals.size}")
# plt.figure(figsize=(8,4))
# parts = plt.violinplot([vis_vals], showmeans=False, showextrema=True)
# for pc in parts.get('bodies', []):
#     pc.set_alpha(0.6)
# # jitter points
# x = np.random.normal(1, 0.04, size=vis_vals.size)
# plt.scatter(x, vis_vals, s=8, alpha=0.6, color='k')
# plt.xticks([1], [f"avg words/turn\n(n={vis_vals.size})"])
# plt.ylabel("avg words per turn")
# plt.title(f"Violin + jitter (values <= {cutoff})")
# plt.ylim(-0.5, cutoff + 1)
# plt.tight_layout()
# plt.show()


In [ ]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import random

def model_family5(name):
    if not name:
        return None
    n = name.lower()
    if n.startswith("gpt-4o"):
        return "gpt-4o"
    if n.startswith("gpt-3.5"):
        return "gpt-3.5-turbo"
    if n.startswith("gpt-4.1"):
        return "gpt-4.1-mini"
    if n.startswith("gpt-4"):
        return "gpt-4"
    if n.startswith("o1"):
        return "o1"
    return None



models = result_ds["model"]
turns  = avg_arr
samples = defaultdict(list)
for m, t in zip(models, turns):
    fam = model_family5(m)
    if fam is None:
        continue
    try:
        val = int(t)
    except Exception:
        continue
    samples[fam].append(val)

order = ["gpt-4o", "gpt-3.5-turbo", "gpt-4.1-mini", "gpt-4", "o1"]

# compute and print robust stats
print("family\tcount\tmedian\tIQR\t10th\t90th\tmean")
for fam in order:
    lst = np.array(samples.get(fam, []))
    if lst.size == 0:
        continue
    med = np.median(lst)
    q1, q3 = np.percentile(lst, [25, 75])
    iqr = q3 - q1
    p10, p90 = np.percentile(lst, [10, 90])
    mean = np.mean(lst)
    print(f"{fam}\t{len(lst)}\t{med:.2f}\t{iqr:.2f}\t{p10:.2f}\t{p90:.2f}\t{mean:.2f}")

# Basic Boxplot
data = []
labels = []
for fam in order:
    lst = samples.get(fam, [])
    if not lst:
        continue
    data.append(lst)
    labels.append(f"{fam}\n(n={len(lst)})")

if not data:
    raise RuntimeError("Fail to collect turn data")

plt.figure(figsize=(12,6))
plt.boxplot(data, tick_labels=labels, showfliers=False)
plt.ylabel("Avg words (int)")
plt.title("Distribution of 'Avg words' by model family")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


# 1) boxplot on log1p scale (recommended for skewed counts)
data_log = [np.log1p(samples[f]) for f in order if samples.get(f)]
labels = [f"{f}\n(n={len(samples[f])})" for f in order if samples.get(f)]

plt.figure(figsize=(10,5))
plt.boxplot(data_log, tick_labels=labels, showfliers=False)
plt.ylabel("log1p(Avg words)")
plt.title("Boxplot (log1p scale) of 'Avg words' by model family")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# 2) violin + jittered scatter on original scale (shows raw spread)
data = [samples[f] for f in order if samples.get(f)]
pos = np.arange(1, len(data) + 1)

plt.figure(figsize=(12,5))
# violin (matplotlib)
parts = plt.violinplot(data, positions=pos, showmeans=False, showextrema=True)
for pc in parts['bodies']:
    pc.set_alpha(0.6)
# jittered scatter
for i, lst in enumerate(data, start=1):
    y = np.array(lst)
    x = np.random.normal(i, 0.08, size=y.size)  # small jitter
    plt.scatter(x, y, s=5, alpha=0.3, color='k')

plt.xticks(pos, labels, rotation=45, ha="right")
plt.ylabel("Avg words (int)")
plt.title("Violin + jittered points (original scale)")
plt.tight_layout()
plt.show()

In [ ]:
# A: turn > cutoff
cutoff = 40
filtered = {f: [v for v in samples.get(f, []) if v <= cutoff] for f in order}
removed_counts = {f: len(samples.get(f, [])) - len(filtered[f]) for f in order}
print("removed (per family) when cutoff=%d:" % cutoff, removed_counts)

data = [filtered[f] for f in order if filtered.get(f)]
labels = [f"{f}\n(n={len(samples.get(f,[]))})" for f in order if filtered.get(f)]

plt.figure(figsize=(12,5))
parts = plt.violinplot(data, positions=np.arange(1, len(data)+1), showmeans=False, showextrema=True)
for pc in parts['bodies']:
    pc.set_alpha(0.6)
# jitter points
for i, lst in enumerate(data, start=1):
    y = np.array(lst)
    if y.size == 0:
        continue
    x = np.random.normal(i, 0.06, size=y.size)
    plt.scatter(x, y, s=8, alpha=0.6, color='k')
plt.xticks(np.arange(1, len(data)+1), labels, rotation=45, ha='right')
plt.ylabel("turn (int)")
plt.title(f"Violin + jitter (turn <= {cutoff})")
plt.ylim(-0.5, cutoff + 2)
plt.tight_layout()
plt.show()

# B: 
data_all = [samples[f] for f in order if samples.get(f)]
labels_all = [f"{f}\n(n={len(samples.get(f,[]))})" for f in order if samples.get(f)]

plt.figure(figsize=(12,5))
parts = plt.violinplot(data_all, positions=np.arange(1, len(data_all)+1), showmeans=False, showextrema=True)
for pc in parts['bodies']:
    pc.set_alpha(0.6)
for i, lst in enumerate(data_all, start=1):
    y = np.array(lst)
    if y.size == 0:
        continue
    x = np.random.normal(i, 0.06, size=y.size)
    plt.scatter(x, y, s=6, alpha=0.5, color='k')
plt.xticks(np.arange(1, len(data_all)+1), labels_all, rotation=45, ha='right')
plt.yscale('log')
plt.ylabel("turn (int), log scale")
plt.title("Violin + jitter (all points, log y-axis)")
plt.tight_layout()
plt.show()

In [ ]:
result_ds["conversation"][0]
print(result_ds.column_names)

## All data

In [ ]:
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

try:
    from dateutil import parser as _dateutil_parser
except Exception:
    _dateutil_parser = None

ts_col = "timestamp"   
batch_size = 10000     

def as_datetime(v):
    if v is None:
        return None
    if isinstance(v, datetime):
        return v
    if isinstance(v, (int, float)):
        return datetime.fromtimestamp(v/1000.0) if abs(v) > 1e11 else datetime.fromtimestamp(v)
    if isinstance(v, str):
        try:
            return datetime.fromisoformat(v)
        except Exception:
            if _dateutil_parser:
                try:
                    return _dateutil_parser.parse(v)
                except Exception:
                    return None
            return None
    return None


print("columns:", train.column_names)
assert ts_col in train.column_names, f"can not find {ts_col}"

counter_all = Counter()
n = len(train)
for start in range(0, n, batch_size):
    end = min(n, start + batch_size)
    batch = train[start:end]
    vals = batch[ts_col]
    for v in vals:
        if isinstance(v, (list, tuple)):
            for sub in v:
                dt = as_datetime(sub)
                if dt:
                    counter_all[dt.date()] += 1
        else:
            dt = as_datetime(v)
            if dt:
                counter_all[dt.date()] += 1

if not counter_all:
    raise RuntimeError("no valid time")

dates_sorted_all = sorted(counter_all.keys())
counts_all = [counter_all[d] for d in dates_sorted_all]

plt.figure(figsize=(12,4))
plt.plot(dates_sorted_all, counts_all, marker='o', linestyle='-')
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation=45)
plt.xlabel('Date')
plt.ylabel('Count')
plt.title('Daily distribution of all timestamps')
plt.tight_layout()
plt.show()

## Topic Analysis
### Frequent Analysis

In [ ]:
def normalize_conv(conv):
    if conv is None:
        return []
    if isinstance(conv, dict):
        # single message dict or nested conversation dict
        if "content" in conv or "text" in conv:
            return [conv]
        conv = conv.get("conversation") or conv.get("conversations")
        if isinstance(conv, dict):
            return [conv]
    if isinstance(conv, (list, tuple)):
        return list(conv)
    return []

def extract_user_texts(dataset, conv_col="conversation", hash_col="conversation_hash"):
    """
    从 dataset 中提取：每条 record 的 conversation 中 role=='user' 的 content/text，
    返回 list of dict: {conversation_hash, turn_index, text}
    """
    out = []
    for record in dataset:
        conv = normalize_conv(record.get(conv_col))
        conv_hash = record.get(hash_col) or record.get("conversation_hash") or None
        for i, turn in enumerate(conv):
            if isinstance(turn, dict):
                role = (turn.get("role") or "").lower()
                text = turn.get("content") or turn.get("text") or ""
                if role == "user" and isinstance(text, str) and text.strip():
                    out.append({"conversation_hash": conv_hash, "turn_index": i, "text": text.strip()})
    return out

user_texts = extract_user_texts(english_ds)   
print("extracted:", len(user_texts))
import pandas as pd
df_user = pd.DataFrame(user_texts)
display(df_user.head())

# out_csv = "english_user_texts.csv"
# df_user[["text", "conversation_hash", "turn_index"]].to_csv(out_csv, index=False, encoding="utf-8-sig")
# print("saved:", out_csv)


In [ ]:
def normalize_conv(conv):
    if conv is None:
        return []
    if isinstance(conv, dict):
        # single message dict or nested conversation dict
        if "content" in conv or "text" in conv:
            return [conv]
        conv = conv.get("conversation") or conv.get("conversations")
        if isinstance(conv, dict):
            return [conv]
    if isinstance(conv, (list, tuple)):
        return list(conv)
    return []
# ...existing code...

def stream_user_texts_to_csv(dataset, out_csv="english_user_texts_stream.csv",
                             conv_col="conversation", hash_col="conversation_hash",
                             show_progress=True):
    """
    使用 dataset.to_iterable_dataset() 流式读取并把 user role 的 content 直接写入 CSV。
    低内存，适合大数据集。
    """
    import csv
    try:
        it = dataset.to_iterable_dataset()
    except Exception:
        it = dataset

    written = 0
    with open(out_csv, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["conversation_hash", "turn_index", "text"])
        for i, record in enumerate(it):
            conv = normalize_conv(record.get(conv_col) or record.get("conversations"))
            conv_hash = record.get(hash_col) or record.get("conversation_hash") or None
            for idx, turn in enumerate(conv):
                if isinstance(turn, dict):
                    role = (turn.get("role") or "").lower()
                    text = turn.get("content") or turn.get("text") or ""
                    if role == "user" and isinstance(text, str) and text.strip():
                        writer.writerow([conv_hash, idx, text.strip()])
                        written += 1
            if show_progress and (i + 1) % 10000 == 0:
                print(f"Processed {i+1} records, written {written} user turns...")
    print(f"Done. total user-turn rows written: {written}. saved to: {out_csv}")
    return out_csv

In [ ]:
# 使用示例（在 notebook cell 中运行）
# 如果 english_ds 支持 iterable：
out = stream_user_texts_to_csv(english_ds, out_csv="english_user_texts_stream.csv")
# 否则使用分块：
# out = chunked_user_texts_to_csv(english_ds, out_csv="english_user_texts_chunks.csv", chunk_size=2000)
# ...existing code...

In [ ]:
def stream_user_texts_to_csv(dataset, out_csv="english_user_texts_stream.csv",
                             conv_col="conversation", hash_col="conversation_hash",
                             show_progress=True, max_user_turns=None, max_records=None):
    """
    流式写入 user role 的 content 到 CSV。
    - max_user_turns: 如果不为 None，最多写入这么多 user-turn（达到后立即停止）。
    - max_records: 如果不为 None，最多扫描这么多 dataset 记录（达到后停止）。
    返回 (out_csv, written_user_turns, scanned_records)
    """
    import csv
    try:
        it = dataset.to_iterable_dataset()
    except Exception:
        it = dataset

    written = 0
    scanned = 0
    with open(out_csv, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["conversation_hash", "turn_index", "text"])
        for i, record in enumerate(it):
            scanned += 1
            conv = normalize_conv(record.get(conv_col) or record.get("conversations"))
            conv_hash = record.get(hash_col) or record.get("conversation_hash") or None
            for idx, turn in enumerate(conv):
                if isinstance(turn, dict):
                    role = (turn.get("role") or "").lower()
                    text = turn.get("content") or turn.get("text") or ""
                    if role == "user" and isinstance(text, str) and text.strip():
                        writer.writerow([conv_hash, idx, text.strip()])
                        written += 1
                        if max_user_turns is not None and written >= int(max_user_turns):
                            if show_progress:
                                print(f"Reached max_user_turns={max_user_turns} after scanning {scanned} records.")
                            print(f"Done. written={written}, scanned={scanned}. saved to: {out_csv}")
                            return out_csv, written, scanned
            if show_progress and (i + 1) % 10000 == 0:
                print(f"Processed {i+1} records, written {written} user turns...")
            if max_records is not None and scanned >= int(max_records):
                if show_progress:
                    print(f"Reached max_records={max_records}.")
                break
    print(f"Done. total user-turn rows written: {written}, scanned records: {scanned}. saved to: {out_csv}")
    return out_csv, written, scanned

In [ ]:
out, written, scanned = stream_user_texts_to_csv(english_ds,
                                                out_csv="D:\\english_user_texts_stream.csv",
                                                max_user_turns=100000,
                                                max_records=50000)
print(out, written, scanned)

In [2]:
import pandas as pd
df = pd.read_csv(r"D:\english_user_texts_stream.csv", encoding="utf-8-sig")
print(df.shape)
display(df.head())

(100000, 3)


,conversation_hash,turn_index,text
0,cf1267ca6b2f6fccc9c36652a00059a1,0,"Old age PT hx of DM, HTN, dyslipidemia His ECG..."
1,59c72510f3143025f94f75b883b026bd,0,i wanna you to write me terms & conditions and...
2,aa7c3f49343e097be66442288abd1dac,0,"Let A, B, and C be events with\n\nProb[A] = 0...."
3,aa7c3f49343e097be66442288abd1dac,2,Question 4 options:\nLet A and B be events wit...
4,aa7c3f49343e097be66442288abd1dac,4,Alice and Bob share binary communication chann...


In [ ]:
# Word frequency (tokenize -> Counter)

import re
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt

CSV_PATH = r"D:\english_user_texts_stream.csv"
USE_CHUNKS = False

# Simple tokenizer
TOKEN_RE = re.compile(r"[A-Za-z0-9']+")

STOPWORDS = {
    'the','and','to','of','a','in','is','it','for','that','on','i','you','be','are',
    'with','this','as','was','but','not','or','have','will','my','we','me','so','if',
    '0','1','2','3','4','5','6','7','8','9','he','she','they','at','by','an','from','all','your',
    'there','what','about','just','like','no','do','when','get','can','would','how','out','up',
    'one','more','some','them','his','her','been','who','now','did','than','then','also','because',
    'into','could','any','other','only','new','these','see','after','over','such','many','much',
    'where','why','those','us','am','too','may','should','well','very','here','most','way','make','even',
    'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
    ',','.','!','?',';','-','--','"',"'",'(',')','[',']','{','}','...',
    'has','had','shall','upon','whereas','therefore','herein','therein','their','its','doth','hath',
    '10','11','12','13','14','15','16','17','18','19','20','its','im','dont','cant','ive','youre','thats',
    'firsr','second','third','also','thus','hence','which','while','during','among','between','within',
    'each','every','either','neither','lest','whilst','moreover','furthermore','additionally'
}

def tokenize(text):
    if not isinstance(text, str): return []
    toks = TOKEN_RE.findall(text.lower())
    return [t for t in toks if t and t not in STOPWORDS]

def freq_from_df(df_iterable):
    cnt = Counter()
    total = 0
    for txt in df_iterable:
        toks = tokenize(txt)
        cnt.update(toks)
        total += len(toks)
    return cnt, total


counter, total_tokens = freq_from_df(df['text'])


TOP_N = 30
top = counter.most_common(TOP_N)
print("Total tokens:", total_tokens)
print("Top", TOP_N, "words:", top[:10])


words, counts = zip(*top) if top else ([],[])
plt.figure(figsize=(12,5))
plt.bar(range(len(counts)), counts, color='C0')
plt.xticks(range(len(words)), words, rotation=45, ha='right')
plt.title(f"Top {TOP_N} words (tokens={total_tokens})")
plt.tight_layout()
plt.show()


In [ ]:
# New cell: extract noun frequencies
import math
from collections import Counter
import matplotlib.pyplot as plt

TOP_N = 40

try:
    import spacy
    nlp = spacy.load("en_core_web_sm", disable=["ner","textcat"])
    def extract_nouns(text):
        if not isinstance(text, str): return []
        doc = nlp(text)
        # 使用 lemma 归一化（小写），保留普通名词和专有名词
        return [tok.lemma_.lower() for tok in doc if tok.pos_ in ("NOUN","PROPN")]
except Exception:
    # 回退到 nltk 的简单规则性 POS 标注（不做词形还原）
    import nltk
    try:
        nltk.data.find("tokenizers/punkt")
    except Exception:
        nltk.download("punkt")
    try:
        nltk.data.find("taggers/averaged_perceptron_tagger")
    except Exception:
        nltk.download("averaged_perceptron_tagger")
    from nltk import word_tokenize, pos_tag
    def extract_nouns(text):
        if not isinstance(text, str): return []
        toks = TOKEN_RE.findall(text.lower())
        tagged = pos_tag(toks)
        return [w for w,tag in tagged if tag.startswith("NN")]

# 统计
noun_counter = Counter()
total_noun_tokens = 0
for txt in df['text']:
    nouns = extract_nouns(txt)
    noun_counter.update(nouns)
    total_noun_tokens += len(nouns)

top_nouns = noun_counter.most_common(TOP_N)
print("Total noun tokens:", total_noun_tokens)
print(f"Top {TOP_N} nouns:", top_nouns[:20])

# 可视化（Top N）
words, counts = zip(*top_nouns) if top_nouns else ([],[])
plt.figure(figsize=(12,5))
plt.bar(range(len(counts)), counts, color='C2')
plt.xticks(range(len(words)), words, rotation=45, ha='right')
plt.title(f"Top {TOP_N} nouns (noun tokens={total_noun_tokens})")
plt.tight_layout()
plt.show()


In [ ]:
# Generate sentence-transformers embeddings for df['text']
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm.auto import tqdm
import torch
import os

MODEL_NAME = "all-MiniLM-L6-v2"
BATCH_SIZE = 256
NORMALIZE = True


device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

TRY_MODELS = [
    "all-MiniLM-L6-v2",
    "sentence-transformers/all-MiniLM-L6-v2",
    "paraphrase-MiniLM-L6-v2",
    "all-mpnet-base-v2"
]

model = None
for mid in TRY_MODELS:
    try:
        model = SentenceTransformer(mid, device=device, trust_remote_code=True)
        print("Loaded model:", mid)
        break
    except Exception as e:
        print("Failed to load", mid, ":", e)
        model = None

if model is None:
    raise RuntimeError("Failed to load any sentence-transformers model. Try upgrading packages and restarting the kernel.")

texts = df['text'].fillna("").astype(str).tolist()

embeddings = model.encode(
    texts,
    batch_size=BATCH_SIZE,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=NORMALIZE
)

print("embeddings shape:", embeddings.shape, "dtype:", embeddings.dtype)

OUT_DIR = r"D:\embeddings_output"
os.makedirs(OUT_DIR, exist_ok=True)
np.save(os.path.join(OUT_DIR, "embeddings.npy"), embeddings)

df_meta = df[["conversation_hash","turn_index"]].reset_index(drop=True)
df_meta.to_csv(os.path.join(OUT_DIR, "embeddings_meta.csv"), index=True)  # index corresponds to embeddings row number

print("Saved embeddings.npy and embeddings_meta.csv to", OUT_DIR)



In [ ]:
import numpy as np
import pandas as pd
from bertopic import BERTopic
import os

OUT_DIR = r"D:\embeddings_output"
EMBEDDINGS_FILE = os.path.join(OUT_DIR, "embeddings.npy")
META_FILE = os.path.join(OUT_DIR, "embeddings_meta.csv")

embeddings = np.load(EMBEDDINGS_FILE)
print(f"Loaded embeddings shape: {embeddings.shape}")


texts = df['text'].fillna("").astype(str).tolist()

c:\Microsoft VS Code\VsCodeProject\Prof.Yu\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded embeddings shape: (100000, 384)


In [ ]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

# 1. Define model UMAP: Used to reduce high-dimensional embeddings to a clusterable dimension
umap_model = UMAP(n_neighbors=20, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

# 2. Define Cludter Model(HDBSCAN):
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# 3. Define Bag-of-words representation (CountVectorizer): Generate c-TF-IDF weights
vectorizer_model = CountVectorizer(stop_words="english") 

# 4. Initialize BERTopic model
topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    nr_topics="auto",
    verbose=True
)

In [5]:
topics, probs = topic_model.fit_transform(texts, embeddings=embeddings)

2025-12-08 22:20:19,433 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-08 22:22:31,749 - BERTopic - Dimensionality - Completed ✓
2025-12-08 22:22:31,752 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-08 22:22:40,362 - BERTopic - Cluster - Completed ✓
2025-12-08 22:22:40,363 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-12-08 22:22:45,753 - BERTopic - Representation - Completed ✓
2025-12-08 22:22:45,779 - BERTopic - Topic reduction - Reducing number of topics
2025-12-08 22:22:45,901 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-08 22:22:51,553 - BERTopic - Representation - Completed ✓
2025-12-08 22:22:51,583 - BERTopic - Topic reduction - Reduced number of topics from 327 to 219


In [ ]:
# 1.  -1 is noise
print("Top 10 Topics:")
print(topic_model.get_topic_info().head(11))

Top 10 Topics:
    Topic  Count                                 Name  \
0      -1  51165            -1_data_public_time_using   
1       0   2620               0_pm_div_center_script   
2       1   2562  1_translate_chinese_english_russian   
3       2   2478     2_stephens_narrative_world_child   
4       3   2082          3_developer_chatgpt_mode_ai   
5       4   1307                   4_hi_hello_hey_uwu   
6       5   1222        5_song_music_secondartist_lil   
7       6   1101               6_pee_diaper_story_pet   
8       7    985                7_line_error_file_pip   
9       8    970            8_security_data_iot_cloud   
10      9    801         9_profit_inflation_stock_qty   

                                       Representation  \
0   [data, public, time, using, use, new, project,...   
1   [pm, div, center, script, padding, 1rem, ul, m...   
2   [translate, chinese, english, russian, gempa, ...   
3   [stephens, narrative, world, child, worldbuild...   
4   [developer,

In [ ]:
topic_id = 4
print(f"\nKeywords for Topic {topic_id}:")
print(topic_model.get_topic(topic_id))


Keywords for Topic 4:
[('hi', np.float64(1.2825488912827983)), ('hello', np.float64(0.5436864288297953)), ('hey', np.float64(0.16399904371564192)), ('uwu', np.float64(0.05107341086500126)), ('doing', np.float64(0.033753380749683995)), ('today', np.float64(0.028076768438267907)), ('hoe', np.float64(0.027959663684613643)), ('helllo', np.float64(0.02178044106495226)), ('clubette', np.float64(0.016876346084525286)), ('uwubot', np.float64(0.016876346084525286))]


In [ ]:
# Visualize the distance between topics
# topic_model.visualize_topics() 
# topic_model.visualize_topics(topics=list(range(10)))

# Visualize the vocabulary of topics
topic_model.visualize_barchart()

In [ ]:
# Save the BERTopic model
MODEL_PATH = os.path.join(OUT_DIR, "bertopic_model")
topic_model.save(MODEL_PATH)
print("\nBERTopic model saved to:", MODEL_PATH)


df['topic'] = topics
df['topic_probability'] = probs
# df.to_csv(os.path.join(OUT_DIR, "df_with_topics.csv"), index=False)

In [ ]:
from bertopic.coherence import CoherenceModel


# achieve topic key word
topic_words = topic_model.get_topics() 

# initialize Coherence Model
coherence_model = CoherenceModel(
    topics=topic_words, 
    texts=texts,
    vectorizer=vectorizer_model,
    coherence='c_v',
    seed=42
)

# count C_v score
coherence_score = coherence_model.get_coherence()

print(f"Topic coherence (C_v Score): {coherence_score:.4f}")

# Count each topic's coherence score
# coherence_per_topic = coherence_model.get_coherence(per_topic=True)
# print("each topic's coherence score C_v:", coherence_per_topic)

ModuleNotFoundError: No module named 'bertopic.coherence'

In [ ]:
# 直接用 gensim 计算 coherence（在 notebook cell 中运行）
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

# texts: list of tokenized docs (list of tokens), e.g. [doc.split() for doc in texts_raw]
tokenized_texts = [t.split() for t in texts]  

# 从 BERTopic 提取每个主题的 top-n 单词
topic_dict = topic_model.get_topics()  # dict: id -> [(word, score), ...]
topic_ids = [tid for tid in topic_dict.keys() if tid != -1]   # 忽略 -1 噪音
top_n = 10
topics_for_gensim = [[w for w,_ in topic_dict[tid][:top_n]] for tid in topic_ids]

# 构建字典并计算 C_v
dictionary = Dictionary(tokenized_texts)
cm = CoherenceModel(topics=topics_for_gensim, texts=tokenized_texts, dictionary=dictionary, coherence='c_v')
score = cm.get_coherence()
print("C_v coherence (gensim):", score)

In [22]:
# 自行计算主题多样性（在 notebook cell 运行）
def compute_topic_diversity(topic_model, top_n=10, ignore_negative_one=True):
    topics = topic_model.get_topics()  # dict: topic_id -> [(word, score), ...]
    all_words = []
    topic_count = 0
    for tid, items in topics.items():
        if ignore_negative_one and tid == -1:
            continue
        if not items:
            continue
        top_words = [w for w,_ in items[:top_n]]
        all_words.extend(top_words)
        topic_count += 1
    if topic_count == 0:
        return 0.0
    unique = len(set(all_words))
    diversity = unique / (top_n * topic_count)
    return diversity

diversity_score = compute_topic_diversity(topic_model, top_n=10)
print(f"主题多样性 (自计算, top_n=10): {diversity_score:.4f}")

主题多样性 (自计算, top_n=10): 0.8849
